In [75]:
import vrep 
import sys
import time 
import numpy as np
from tank import *
from skfuzzy import control as ctrl

## Połączenie z V-REP

In [76]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)

#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)


Connected to remote API server


In [77]:
def read_proximity_sensor(sensor_name: str):
    sensors = {
        "EN": 0,
        "ES": 1,
        "NE": 2,
        "NW": 3,
        "SE": 4,
        "SW": 5,
        "WN": 6,
        "WS": 7
    }
    _,detectionState,detectedPoint,_,_=vrep.simxReadProximitySensor(clientID,proximity_sensors_handles[sensors[sensor_name]],vrep.simx_opmode_buffer)
    return detectionState, detectedPoint

In [80]:
x_speeds = np.arange(-3, 8, 1)
x_distances = np.arange(2, 3.6, 0.1)

NW_dist = ctrl.Antecedent(x_distances, 'NW_dist')
WN_dist = ctrl.Antecedent(x_distances, 'WN_dist')
target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

NW_dist.automf(3)
WN_dist.automf(3)
target_speed.automf(5)

rules = (
    ctrl.Rule(NW_dist['good'] & WN_dist['poor'], target_speed['mediocre']),
    ctrl.Rule((NW_dist['poor'] | NW_dist['good']) & (WN_dist['average'] | WN_dist['average']), target_speed['good']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    detectionState, NW_detectedPoint = read_proximity_sensor("NW")
    if detectionState:
        NW_curr_dist = np.linalg.norm(NW_detectedPoint)
    else:
        NW_curr_dist = 100

    detectionState, WN_detectedPoint = read_proximity_sensor("WN")
    if detectionState:
        WN_curr_dist = np.linalg.norm(WN_detectedPoint)
    else:
        WN_curr_dist = 100

    driver.input['NW_dist'] = NW_curr_dist
    driver.input['WN_dist'] = WN_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values NW:{NW_curr_dist}')
        break

    target = driver.output['target_velocity']

    print(f'\rNW_dist:{NW_curr_dist:.2f} WN_dist: {WN_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        target = 0
        tank.forward(target)
        break

    tank.forward(target)

NW_dist:3.51 WN_dist: 2.09, target_velocity: 0.24

In [82]:
x_speeds = np.arange(-3, 8, 1)
x_distances = np.arange(1, 3, 0.1)

NW_dist = ctrl.Antecedent(x_distances, 'NW_dist')
NE_dist = ctrl.Antecedent(x_distances, 'NE_dist')
target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

NE_dist.automf(3)
NW_dist.automf(3)
target_speed.automf(5)

rules = (
    ctrl.Rule((NE_dist['average'] & NW_dist['average']) |
              (NE_dist['poor'] & NW_dist['poor']), target_speed['mediocre']),
    ctrl.Rule(NW_dist['good'] | NE_dist['good'] |
              (NE_dist['average'] & NW_dist['poor']) |
              (NE_dist['poor'] & NW_dist['average']), target_speed['good']),
)

driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    detectionState, NW_detectedPoint = read_proximity_sensor("NW")
    if detectionState:
        NW_curr_dist = np.linalg.norm(NW_detectedPoint)
    else:
        NW_curr_dist = 100

    detectionState, NE_detectedPoint = read_proximity_sensor("NE")
    if detectionState:
        NE_curr_dist = np.linalg.norm(NE_detectedPoint)
    else:
        NE_curr_dist = 100

    driver.input['NW_dist'] = NW_curr_dist
    driver.input['NE_dist'] = NE_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values NW_dist:{NW_curr_dist:.2f} NE_dist: {NE_curr_dist:.2f}')
        break

    target = driver.output['target_velocity']

    print(f'\rNW_dist:{NW_curr_dist:.2f} NE_dist: {NE_curr_dist:.2f}, target_velocity: {target:.2f}', end='')
    if target <= 0.5:
        target = 0
        tank.forward(target)
        break

    tank.turn_left(1.3 * target)
    tank.forward(target)

NW_dist:1.15 NE_dist: 1.15, target_velocity: 0.4010

In [84]:
x_speeds = np.arange(-2, 5, 1)
x_distances = np.arange(1, 3, 0.1)


NE_dist = ctrl.Antecedent(x_distances, 'NE_dist')
WN_dist = ctrl.Antecedent(x_distances, 'WN_dist')
EN_dist = ctrl.Antecedent(x_distances, 'EN_dist')
target_speed = ctrl.Consequent(x_speeds, 'target_velocity')

NE_dist.automf(3)
EN_dist.automf(3)
WN_dist.automf(3)
target_speed.automf(5)
rules = (
    ctrl.Rule(WN_dist['poor'] & EN_dist['poor'], target_speed['poor']),
    ctrl.Rule(NE_dist['average'] & EN_dist['average'], target_speed['poor']),
    ctrl.Rule(NE_dist['poor'], target_speed['good']),
    ctrl.Rule(NE_dist['good'], target_speed['mediocre']),
)


driver = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules=rules))

while True:
    detectionState, NW_detectedPoint = read_proximity_sensor("NW")
    if detectionState:
        NW_curr_dist = np.linalg.norm(NW_detectedPoint)
    else:
        NW_curr_dist = 100

    detectionState, NE_detectedPoint = read_proximity_sensor("NE")
    if detectionState:
        NE_curr_dist = np.linalg.norm(NE_detectedPoint)
    else:
        NE_curr_dist = 100

    detectionState, WN_detectedPoint = read_proximity_sensor("WN")
    if detectionState:
        WN_curr_dist = np.linalg.norm(WN_detectedPoint)
    else:
        WN_curr_dist = 100

    detectionState, EN_detectedPoint = read_proximity_sensor("EN")
    if detectionState:
        EN_curr_dist = np.linalg.norm(EN_detectedPoint)
    else:
        EN_curr_dist = 100

    driver.input['NE_dist'] = NE_curr_dist
    driver.input['WN_dist'] = WN_curr_dist
    driver.input['EN_dist'] = EN_curr_dist

    try:
        driver.compute()
    except:
        print(f'Error for values NW_dist:{NW_curr_dist:.2f} NE_dist: {NE_curr_dist:.2f} WN_dist: {WN_curr_dist:.2f} EN_dist: {EN_curr_dist:.2f}')
        break

    target = driver.output['target_velocity']

    print(f'\r NW_dist: {NW_curr_dist:.2f} NE_dist: {NE_curr_dist:.2f} WN_dist: {WN_curr_dist:.2f} EN_dist: {EN_curr_dist:.2f}, target_velocity: {target:.2f}', end='')

    if target <= 0:
        target = 0
        tank.stop()
        break

    tank.turn_left(target)
    tank.forward(1)

 NW_dist: 0.64 NE_dist: 100.00 WN_dist: 0.44 EN_dist: 0.58, target_velocity: -0.82